In [1]:
import pandas as pd
import numpy as np

In [2]:
file = "data/exoplanet.eu_catalog.csv"

In [3]:
data = pd.read_csv(file, skiprows=0, usecols=['radius', 'orbital_period', 'ra', 'dec', 'mag_v', 
                                              'star_distance', 'star_radius', 'semi_major_axis',
                                              'eccentricity', 'tzero_tr'])

In [4]:
# We want to drop rows with null planet and stellar radii
# We also need the V-band magnitudes
data = data.dropna(subset={"radius", "star_radius", "mag_v", "tzero_tr"})
# We want only stars with mag_v ~< 12
data = data[data["mag_v"] < 12]

# Now compute transit depth for each
transit_depth = (0.10049*data["radius"] / data["star_radius"])**2
# Compute transit signature in magnitudes
transit_sig = -2.5 * np.log10(1 - transit_depth)
# Drop all rows where transit signature is < 0.008
data = data[transit_sig > 0.008]

In [5]:
semi_minor_axis = data["semi_major_axis"] * np.sqrt(1 - data["eccentricity"]**2)
# This is the lower bound on circumference
circ = np.sqrt(2) * np.pi * np.sqrt(data["semi_major_axis"]**2 + semi_minor_axis**2)
# Velocity in meters per hour
veloc = circ / data["orbital_period"] * 1.496e11 / 24
# Very rough duration estimate
transit_dur = 2 * data["star_radius"] * 6.957e8 / veloc
# Keep transit duration under 3 hours
data = data[transit_dur < 3]

In [6]:
data = data[data["dec"] > -19]

ra = (data["ra"] + 180) % 360
data = data[ra > 285]

In [7]:
# September 5th at 19:00
after = 2458731.29167
# Compute the next transit
rem = (after - data["tzero_tr"]) % data["orbital_period"]
obsday = after - rem + data["orbital_period"]

for i in obsday.index:
    
    while not (0.3 <= obsday[i] % 1 <= 0.5): obsday[i] += data["orbital_period"][i]
        
print(*obsday)

2458735.4547449993 2458746.34906 2458737.300653599 2458733.313034


In [8]:
data

,radius,orbital_period,semi_major_axis,eccentricity,tzero_tr,ra,dec,mag_v,star_distance,star_radius
326,0.867,2.875317,0.03610,0.015,2.456708e+06,111.916667,24.336389,11.34,70.0,0.694
328,1.080,3.212220,0.04140,0.016,2.456707e+06,155.683333,50.128333,9.73,82.0,1.040
3856,1.137,1.755414,0.02918,0.000,2.456406e+06,160.602542,7.435083,11.12,143.0,0.930
4048,0.973,2.615838,0.03690,0.045,2.454148e+06,117.029167,50.225833,11.18,149.0,0.964
